# Example of combined use of bindome and greatpy 

This notebook shows how to combine the bindome package and greatpy. For this it is necessary to have a file annotations which is in /storage/groups/ml01/datasets/. 

The test will be done on the SRF file in order to obtain the same results as in the publication of great:  

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import greatpy as great
import bindome as bd
import warnings
warnings.filterwarnings('ignore')

In [3]:
annpath = "/home/tom/Stage/annotation"
# annpath='/storage/groups/ml01/datasets/annotations'
bd.constants.ANNOTATIONS_DIRECTORY = annpath
df1 = bd.bindome.datasets.REMAP2020.get_remap_peaks('SRF')
df1

,chr,start,end,3,4,5,6,7,8,coordinate,summit.start,summit.end,k.summit
0,chr1,629831,629954,SRF:K-562,1,.,629938,629939,"140,112,224",chr1:629831-629954,629838,630038,chr1:629838-630038
1,chr1,778597,778779,"SRF:Ishikawa,GM12878,A-673-clone-Asp114,K-562,...",8,.,778671,778672,"140,112,224",chr1:778597-778779,778571,778771,chr1:778571-778771
2,chr1,869817,870104,SRF:A-673-clone-Asp114,1,.,870004,870005,"140,112,224",chr1:869817-870104,869904,870104,chr1:869904-870104
3,chr1,904714,904924,"SRF:GM12878,A-673-clone-Asp114",2,.,904795,904796,"140,112,224",chr1:904714-904924,904695,904895,chr1:904695-904895
4,chr1,905143,905666,SRF:A-673-clone-Asp114,1,.,905393,905394,"140,112,224",chr1:905143-905666,905293,905493,chr1:905293-905493
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130753,chrY,19567211,19567311,SRF:Hep-G2,1,.,19567231,19567232,"140,112,224",chrY:19567211-19567311,19567131,19567331,chrY:19567131-19567331
130754,chrY,19744932,19745043,SRF:Hep-G2,1,.,19744983,19744984,"140,112,224",chrY:19744932-19745043,19744883,19745083,chrY:19744883-19745083
130755,chrY,20575583,20575822,"SRF:WA01,Hep-G2",2,.,20575707,20575708,"140,112,224",chrY:20575583-20575822,20575607,20575807,chrY:20575607-20575807
130756,chrY,20575934,20576114,"SRF:WA01,Hep-G2",2,.,20576046,20576047,"140,112,224",chrY:20575934-20576114,20575946,20576146,chrY:20575946-20576146


Path should be modify when I have access to servers

In [4]:
import pandas as pd
name = pd.unique(list(df1[3]))
for n in name : 
    if df1[df1[3]==n].shape[0] > 450 and df1[df1[3]==n].shape[0] < 2000 : 
        print(df1[df1[3]==n].shape[0],n)

1010 SRF:Ishikawa,GM12878,A-673-clone-Asp114,K-562,WA01,MCF-7,HCT-116,Hep-G2
1416 SRF:GM12878,A-673-clone-Asp114
629 SRF:Ishikawa,GM12878
578 SRF:Ishikawa,MCF-7
711 SRF:A-673-clone-Asp114,K-562
959 SRF:A-673-clone-Asp114,Hep-G2
1149 SRF:WA01
646 SRF:A-673-clone-Asp114,HCT-116


Find the biosample used in the publication of great (556 genomic regions)

In [5]:
df1 = df1[df1[3]=="SRF:Ishikawa,MCF-7"].iloc[:,0:3]
df1 = df1.rename(columns={"chr":'Chr','start':"Chr_start",'end':"Chr_end"})
df1

,Chr,Chr_start,Chr_end
47,chr1,1157504,1157660
178,chr1,2586482,2586614
241,chr1,3900438,3900894
371,chr1,6785102,6785311
708,chr1,10796590,10796818
...,...,...,...
130119,chrX,132023226,132023393
130187,chrX,133418786,133418958
130266,chrX,135344596,135344830
130562,chrX,152830686,152830951


In [6]:
df1.to_csv('~/Stage/bindome_df.bed',sep='\t', index=False,header=False, columns=['Chr','Chr_start','Chr_end'])

test to try to understand the computing time problem

In [7]:
res = great.tl.enrichment(
    test = df1,
    regdom_file="~/Stage/greatpy/data/human/regulatory_domain.bed",
    chr_size_file="~/Stage/greatpy/data/human/chr_size.bed",
    annotation="~/Stage/greatpy/data/human/ontologies.csv",
    binom=True,
    hypergeom=True,
    alpha=0.05,
    correction=(0,0),
    )

In [15]:
res

,go_term,binom_p_value,hypergeom_p_value
GO:0070268,cornification,2.16672e-284,NaN


9 min to analysis 500 genomic region with greatpy.tl.enrichment but the result is not the same as great

In [10]:
%lprun -f great.tl.enrichment great.tl.enrichment(df1,"~/Stage/greatpy/data/human/regulatory_domain.bed","~/Stage/greatpy/data/human/chr_size.bed","~/Stage/greatpy/data/human/ontologies.csv",True,True,0.05,(0,0))

Timer unit: 1e-06 s

Total time: 763.606 s
File: /home/tom/.local/lib/python3.8/site-packages/greatpy/tl/basic.py
Function: enrichment at line 318

Line #      Hits         Time  Per Hit   % Time  Line Contents
   318                                           def enrichment(test:str or pd.DataFrame,regdom_file,chr_size_file,annotation,binom=True,hypergeom=True,alpha=0.05,correction=("fdr",0.05),sort_by=None): 
   319                                               # Data import 
   320         1         79.0     79.0      0.0      if not binom and not hypergeom : 
   321                                                   return False
   322                                               
   323         2      51765.0  25882.5      0.0      regdom = pd.read_csv(regdom_file,sep="\t",comment="#",
   324         1          4.0      4.0      0.0                      names=["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"